# Radial Basics Functions a Recurrent Neural Network 

Minule jsme si ukazovali a zkoušeli si naprogramovat jednoduché neuronové sítě. Dneska se podíváme na složitější struktury neuronových sítí Radial Basics Functions a rekurentní neuronové sítě.


## Radial Basics Functions (RBF)

Naimplementujeme si nyní RBF síť. Implementace v tensorflow je jednoduchá -- stačí naimplementovat třídu, která bude mít dvě metody. Metoda *build* nainicializuje parametry podle velikosti vstupu a metoda *call* pak implementuje vlastní výpočet.

In [1]:
import tensorflow as tf
from sklearn import datasets
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import train_test_split
import numpy as np
import random
import sys

C:\Users\katie\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class RBFNetwork():
    def __init__(self, input_dim, num_centers, output_dim):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_centers = num_centers
        self.centers = [np.random.uniform(-1, 1, input_dim) for i in range(num_centers)]
        self.beta = 1
        self.weights = None
        
    # spocteni hodnot neuronu na skryte vrstve
    def calculate_activation(self, data):
        hidden_layer_values = np.zeros((data.shape[0], self.num_centers), float)
        for c_idx, c in enumerate(self.centers):
            for x_idx, x in enumerate(data):
                hidden_layer_values[x_idx,c_idx] = self.activation_fcn(c, x)
        return hidden_layer_values
    
    #  hodnota aktivacni fce
    def activation_fcn(self, centers, data):
        return np.exp(-self.beta * np.linalg.norm(centers-data)**2)
    
    def fit(self, data, labels):
        # zvol nahodne hodnoty pocatecnich centroidu
        random_idx = np.random.permutation(data.shape[0])[:self.num_centers]
        self.centers = [data[i,:] for i in random_idx]
         
        # spocitame aktivaci mezi vstupem a skrytou vrstvou
        hidden_layer_values = self.calculate_activation(data)
         
        # porovname skutecne a predikovane vystupy a aktualizujem vahy 
        # pseudoinverzni matice je vlastne vzorecek pro LR pro train vah
        self.weights = np.dot(np.linalg.pinv(hidden_layer_values), labels)
          
    def predict(self, data):
        hidden_layer_values = self.calculate_activation(data)
        labels = np.dot(hidden_layer_values, self.weights)
        return labels


Zkusíme si naši RBF síť pustit na našem oblíbeném datasetu Iris. Načteme si data a labely, do které třídy data patří. Protože budeme dělat klasifikaci, je vhodné si labely převést na one-hot-encoding. Následně data rozdělíme na trénovací a testovací množinu, abychom mohli zvlášť data trénovat a zvlášť data testovat.

In [3]:
# ziskame data
iris = datasets.load_iris()
x, y = iris.data, iris.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# delame klasifikaci prevedeme vystup do one hot encoding
onehotencoder = OneHotEncoder(categorical_features = [0]) 
y_train_onehot = onehotencoder.fit_transform(y_train.reshape(-1, 1)).toarray() 
y_test_onehot = onehotencoder.fit_transform(y_test.reshape(-1, 1)).toarray() 

# natrenujeme a ohodnotime sit
rbf = RBFNetwork(4, 10, 3)
rbf.fit(x_train, y_train_onehot)
predicted = rbf.predict(x_test)
y_pred = np.argmax(predicted, axis=1)
accuracy = np.mean(y_pred == y_test)
print('Accuracy: ' + str(accuracy))

Accuracy: 0.9473684210526315


Vidíme, že accuracy je v každém běhu dosti různá, což je způsobeno tím, že centroidy a betu zde nastavujeme náhodně, přestože jsme si říkali, že pozice centroidů se dá trénovat pomocí algoritmu k-means.

### Úkol na cvičení

Zkuste si naimplementovat algoritmus k-means pro inicializaci středů vstupních neuronů a zlepšit tím výstup sítě výše. Hint: metoda `add_variable` má parametr initializer.


## Rekurentní neuronové sítě

Rekurentní neuronová síť je síť, která navíc ke svému vstupu ještě bere jako další vstup svůj výstup z předchozího kroku. Proto výstupy z předchozí vrstvy mohou ovlivňovat vrstvu následující, což se může hodit například u generování časových řad nebo textu. Nejprve se podíváme, jak přesně by vypadala implementace jednoduché RNN, kdybychom si ji psali celou sami. 

Vytvoříme si nejprve jednoduchá v2ěty a budeme chtít, aby nám naše sít uměla predikovat, zda je daná věta pozitivní nebo negativní. 

In [4]:
train_data = {
  'good': True,
  'bad': False,
  'happy': True,
  'sad': False,
  'not good': False,
  'not bad': True,
  'not happy': False,
  'not sad': True,
  'very good': True,
  'very bad': False,
  'very happy': True,
  'very sad': False,
  'i am happy': True,
  'this is good': True,
  'i am bad': False,
  'this is bad': False,
  'i am sad': False,
  'this is sad': False,
  'i am not happy': False,
  'this is not good': False,
  'i am not bad': True,
  'this is not sad': True,
  'i am very happy': True,
  'this is very good': True,
  'i am very bad': False,
  'this is very sad': False,
  'this is very happy': True,
  'i am good not bad': True,
  'this is good not bad': True,
  'i am bad not good': False,
  'i am good and happy': True,
  'this is not good and not happy': False,
  'i am not at all good': False,
  'i am not at all bad': True,
  'i am not at all happy': False,
  'this is not at all sad': True,
  'this is not at all happy': False,
  'i am good right now': True,
  'i am bad right now': False,
  'this is bad right now': False,
  'i am sad right now': False,
  'i was good earlier': True,
  'i was happy earlier': True,
  'i was bad earlier': False,
  'i was sad earlier': False,
  'i am very bad right now': False,
  'this is very good right now': True,
  'this is very sad right now': False,
  'this was bad earlier': False,
  'this was very good earlier': True,
  'this was very bad earlier': False,
  'this was very happy earlier': True,
  'this was very sad earlier': False,
  'i was good and not bad earlier': True,
  'i was not good and not happy earlier': False,
  'i am not at all bad or sad right now': True,
  'i am not at all good or happy right now': False,
  'this was not happy and not good earlier': False,
}

test_data = {
  'this is happy': True,
  'i am good': True,
  'this is not happy': False,
  'i am not good': False,
  'this is not bad': True,
  'i am not sad': True,
  'i am very good': True,
  'this is very bad': False,
  'i am very sad': False,
  'this is bad not good': False,
  'this is good and happy': True,
  'i am not good and not happy': False,
  'i am not at all sad': True,
  'this is not at all good': False,
  'this is not at all bad': True,
  'this is good right now': True,
  'this is sad right now': False,
  'this is very bad right now': False,
  'this was good earlier': True,
  'i was not happy and not good earlier': False,
}

Nejprve je potřeba udělat nějaký preprocessing vět. Tím se myslí převést text do číselné reprezentace. To uděláme tak, že najdeme všechno unikátní slova, očíslujeme je, a každé slovo pak nahradíme jeho číslem. Následně uděláme one-hot-encoding každého slova (a to potom bude mít shape (vocab_size, 1)). 

In [5]:
# vytvorime si slovnik veskerych slov z dat
vocab = list(set([w for text in train_data.keys() for w in text.split(' ')]))
vocab_size = len(vocab)
print(vocab)
print('Unique words: ' + str(vocab_size))

# kazde slovo nahradime jeho indexem
word_to_idx = { w: i for i, w in enumerate(vocab) }
idx_to_word = { i: w for i, w in enumerate(vocab) }
print(word_to_idx['happy']) 
print(idx_to_word[0]) 

# prevedeme jeste slova do one hot reprezentace 
def create_inputs(text):
    inputs = []
    for w in text.split(' '):
        v = np.zeros((vocab_size, 1))
        v[word_to_idx[w]] = 1
        inputs.append(v)
    return inputs

['am', 'all', 'right', 'earlier', 'good', 'sad', 'happy', 'very', 'this', 'and', 'at', 'not', 'i', 'now', 'was', 'bad', 'or', 'is']
Unique words: 18
6
am


Nyní si napíšeme samotnou RNN. Informace z předchozího kroku se ukládá v RNN buňce do speciální proměnné -- tzv. skrytého stavu, který nám pak bude ovlivňovat další výstupy a bude se s každým novým vstupem v každém kroku aktualizovat. Aktuální krytý stav se počítá podle předchozího skrytého stavu a aktuálního vstupu. Výstup je spočítán pomocí aktuálního skrytého stavu. Pro každý krok se používají stejné 3 váhové matice: pro spoje z aktuálních vstupů do aktuálních skrytých, pro spoje z předchozích skrytých do aktuálních skrytých a pro spoje z aktuálních skrytých do vystupů. Zároveň potřebujeme bias pro spočtení skrytého stavu a další pro spočtení výstupu. Pomocí aktivační funkce tanh a dosazení hodnot do rovnice umíme vypočítat výstup a update skrytých stavů.

In [12]:
class RNN:
    def __init__(self, input_size, output_size, hidden_size=64):
        # vahy, deleni je kvuli snizeni variance 
        self.Whh = np.random.randn(hidden_size, hidden_size)/1000
        self.Wxh = np.random.randn(hidden_size, input_size)/1000
        self.Why = np.random.randn(output_size, hidden_size)/1000

        # biasy
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))
        
    def forward(self, inputs):
        # zapamatovani si hodnoty predchoziho skryteho stavu
        h = np.zeros((self.Whh.shape[0], 1))

        # postupny update skrytého stavu
        for i, x in enumerate(inputs):
            h = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h) + self.bh)

        # vypocet vystupniho vektoru
        y = np.dot(self.Why, h) + self.by
        return y, h

In [13]:
# definujeme si fci softmax, na namapovani hodnot do intervalu [0,1]  
def softmax(x):
    return np.exp(x) / sum(np.exp(x))

# inicializace RNN
inputs = create_inputs('i am very good')
rnn = RNN(vocab_size, 2)
y, h = rnn.forward(inputs)
probs = softmax(y)
print(probs)

[[0.49999536]
 [0.50000464]]


Vidíme, že síť nám sice nějak funguje, ale není moc užitečná. Problém je, že nijak netrénujeme váhy. K tomu je potřeba si definovat ztrátovou funkci, použijeme cross-entropy loss, která se spočítá jako mínus logaritmus počtu správně predikovaných tříd.  Zároveň je potřeba dopsat zpětnou propagaci chyby, aby se síť mohla učit ze svých chyb a updatovat si váhy a skryté stavy. To je v podstatě jen derivace tanh, dosazení do vzorečků a použití řetízkového pravidla.

In [16]:
class RNN:
    def __init__(self, input_size, output_size, hidden_size=64):
        # vahy, deleni je kvuli snizeni variance 
        self.Whh = np.random.randn(hidden_size, hidden_size)/1000
        self.Wxh = np.random.randn(hidden_size, input_size)/1000
        self.Why = np.random.randn(output_size, hidden_size)/1000

        # biasy
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))
        
    def forward(self, inputs):
        # zapamatovani si hodnoty predchoziho skryteho stavu
        h = np.zeros((self.Whh.shape[0], 1))

        self.last_inputs = inputs
        self.last_hs = { 0: h }

        # postupny update skrytého stavu
        for i, x in enumerate(inputs):
            h = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h) + self.bh)
            self.last_hs[i + 1] = h

        # vypocet vystupniho vektoru
        y = np.dot(self.Why, h) + self.by

        return y, h
    
    def backprop(self, probs, target, learn_rate=2e-2):
        # spocitame derivaci na vystupu podle chyby dL/dy
        d_y = probs
        d_y[target] -= 1
        n = len(self.last_inputs)

        # spocitame derivaci na vystupu z chyby pro vahy dL/dWhy a bias dL/dby 
        d_Why = np.dot(d_y, self.last_hs[n].T)
        d_by = d_y

        # inicializujeme si nulove dL/dWhh, dL/dWxh, a dL/dbh, budeme si do nich pocitat chybu 
        d_Whh = np.zeros(self.Whh.shape)
        d_Wxh = np.zeros(self.Wxh.shape)
        d_bh = np.zeros(self.bh.shape)

        # spocteme dL/dh for pro posledni vystup h
        d_h = np.dot(self.Why.T, d_y)

        # backpropagujeme chybu zpet v case pomoci dosazovani do rovnic
        for t in reversed(range(n)):
            # pomocna hodnota: dL/dh * (1 - h^2)
            temp = ((1 - self.last_hs[t + 1] ** 2) * d_h)

            # dL/db = dL/dh * (1 - h^2)
            d_bh += temp

            # dL/dWhh = dL/dh * (1 - h^2) * h_{t-1}
            d_Whh += np.dot(temp, self.last_hs[t].T)

            # dL/dWxh = dL/dh * (1 - h^2) * x
            d_Wxh += np.dot(temp, self.last_inputs[t].T)

            # dL/dh = dL/dh * (1 - h^2) * Whh
            d_h = np.dot(self.Whh, temp)

        # abychom zabranili pirlis velkym gradientum, omezime hodnoty do intervalu [-1,1]
        for d in [d_Wxh, d_Whh, d_Why, d_bh, d_by]:
                  np.clip(d, -1, 1, out=d)

        # update vah a biasu pomoci gradient descent
        self.Whh -= learn_rate * d_Whh
        self.Wxh -= learn_rate * d_Wxh
        self.Why -= learn_rate * d_Why
        self.bh -= learn_rate * d_bh
        self.by -= learn_rate * d_by

In [17]:
def run_model(data, train):
    items = list(data.items())
    random.shuffle(items)

    loss = 0
    correct_answers = 0

    for x, y in items:
        inputs = create_inputs(x)
        target = int(y)

        # dopredny pruchod
        out, _ = rnn.forward(inputs)
        probs = softmax(out)

        # spocitani loss/accuracy
        loss -= np.log(probs[target])
        correct_answers += int(np.argmax(probs) == target)

        if train==True:
            # pzetny pruchod
            rnn.backprop(probs, target)

    return loss/len(data), correct_answers/len(data)

rnn = RNN(vocab_size, 2)

for epoch in range(1001):
    train_loss, train_accuracy = run_model(train_data, train=True)
    if epoch % 100 == 0:
        print('--- Epoch %d' % (epoch))
        print('Train:\tLoss %.3f | Accuracy: %.3f' % (train_loss, train_accuracy))

        test_loss, test_accuracy = run_model(test_data, train=False)
        print('Test:\tLoss %.3f | Accuracy: %.3f' % (test_loss, test_accuracy))

--- Epoch 0
Train:	Loss 0.696 | Accuracy: 0.517
Test:	Loss 0.694 | Accuracy: 0.500
--- Epoch 100
Train:	Loss 0.688 | Accuracy: 0.552
Test:	Loss 0.698 | Accuracy: 0.500
--- Epoch 200
Train:	Loss 0.665 | Accuracy: 0.655
Test:	Loss 0.723 | Accuracy: 0.450
--- Epoch 300
Train:	Loss 0.573 | Accuracy: 0.690
Test:	Loss 0.627 | Accuracy: 0.700
--- Epoch 400
Train:	Loss 0.406 | Accuracy: 0.828
Test:	Loss 0.735 | Accuracy: 0.600
--- Epoch 500
Train:	Loss 0.373 | Accuracy: 0.828
Test:	Loss 0.714 | Accuracy: 0.600
--- Epoch 600
Train:	Loss 0.318 | Accuracy: 0.828
Test:	Loss 0.869 | Accuracy: 0.600
--- Epoch 700
Train:	Loss 0.374 | Accuracy: 0.810
Test:	Loss 0.428 | Accuracy: 0.700
--- Epoch 800
Train:	Loss 0.416 | Accuracy: 0.759
Test:	Loss 0.337 | Accuracy: 0.850
--- Epoch 900
Train:	Loss 0.008 | Accuracy: 1.000
Test:	Loss 0.014 | Accuracy: 1.000
--- Epoch 1000
Train:	Loss 0.003 | Accuracy: 1.000
Test:	Loss 0.005 | Accuracy: 1.000


##  Sekvenční klasifikace pomocí LSTM

Nyní, když chápeme, jak taková základní RNN funguje, zkusíme se podívat na složitější druh RNN -- LSTM sítě. Tyto sítě mají navíc uvnitř sebe paměťovou buňku a mechanizmus, který řídí,  jakou informaci si buňka pamatuje a jakou zapomíná. Zkusíme se na ni lépe podívat v následujícím příkladu sekvenční klasifikace. 

Sekvenční klasifikace je prediktivní modelovací problém, kdy máme na vstupu nějakou sekvenci v prostoru nebo čase a cílem je předpovědět kategorii této sekvence. Složitost tohoto problému spočívá v tom, že jednotvlivé sekvence mohou mít různou délku nebo mohou být složeny z rozsáhlého slovníku vstupních hodnot a mohou vužadovat, aby se model naučil nějaké dlouhodobé závislosti nebo kontext mezi vstupními sekvencemi.

Zkusíme se tedy podívat na příklad sekvenční klasifikace pomocí LSTM na IMDB datasetu, což je dataset, který obsahuje slovní popis recenzí 50K filmů a následně klasifikaci, jestli byla recenze pozitivní nebo negativní v poměru 1:1. 

In [31]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# zafixujem random seed pro reprodukovatelnost
numpy.random.seed(7)

Problém je, že slovní popis je nějak potřeba převést na číselnou reprezentaci. Naštěstí funkce ```imdb.load_data``` umí načíst data tak, že rovnou slova nahradí čísly a rozdělí je na train a test množiny v poměru 1:1. Navíc data načteme tak, že necháme jen prvních ```top_words``` nejčastějších slova  zbytek nahradíme 0. Dále je potřeba zkrátit nebo doplnit vstupní sekvence pro modelování tak, aby byly všechny stejně dlouhé, délku nastavíme na ```max_len```.

In [28]:
top_words = 5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_words)

max_length = 500
x_train = sequence.pad_sequences(x_train, maxlen=max_length)
x_test = sequence.pad_sequences(x_test, maxlen=max_length)

Nyní máme připravená data a můžeme si definovat a natrénovat model.
 - První vrstva je ```Embedding```, která používá vektory délky 32 pro každé slovo. 
 - Další vrstva je ```LSTM``` vrstva, která obsahuje 100 paměťových jednotek (neuronů). 
 - Na závěr použijeme ```Dense``` výstupní vrstvu s jedním neuronem a aktivační funkcí sigmoid k vytvoření predikcí 0 nebo 1, protože se jedná o klasifikační úlohu.

Problém modelu je, že se velice snadno overfittuje na na daná trénovací data. Proto se se používají ještě vrstvy ```Dropout```, které spočívají v tom, že během trénování se náhodně vynechávají některé vstupy do další vrstvy. Tím se simuluje velký počet sítí s odlišnou strukturou a uzly jsou pak robustnější a omezuje se tím pádem overfitting.

In [32]:
# create the model
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, epochs=3, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
25000/25000 [==============================] - 484s 19ms/step - loss: 0.4778 - acc: 0.7633
Epoch 2/3
25000/25000 [=======

Na závěr zkusíme predikovat výstupy na testovacích datech a podívat se, jak je model dobrý. Můžete si třeba zkusit pustit trénování modelu s droupoutem a bez něj a podívat se, jak se budou lišit výsledné accuracies.

In [33]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 87.14%


## Generování textu znak po znaku 

Nyní se podíváme na jiný druh problému -- budeme generovat text znak po znaku, neboli natrénujeme jazykový model tak, že když mu pak dáme sekvenci znaků, tak nám model bude schopný předpovědět další znak. Jako trénovací množinu použijeme texty Nietzscheho.

In [ ]:
'''
    Example script to generate text from Nietzsche's writings.
    At least 20 epochs are required before the generated text
    starts sounding coherent.
    It is recommended to run this script on GPU, as recurrent
    networks are quite computationally intensive.
    If you try this script on new data, make sure your corpus
    has at least ~100k characters. ~1M is better.
'''

# nacteme si vstupni data
path = tf.keras.utils.get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

chars = set(text)
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# rozdelime text na castecne zavisle sekvence znaku delky maxlen
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

# prevedeme text na ciselne vektory
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
# vytvorime si model    
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars), activation=tf.nn.softmax))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# pomocna funkce k ziskani indexu z pravdepodobnostniho pole
def sample(a, temperature=1.0):  
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    a = a/np.sum(a)
    return np.argmax(np.random.multinomial(1, a, 1))

# natrenujeme model a po kazde iterace generujeme vystup
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for _ in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Skript si samozřejmě můžete pustit, ale trénování poběží neskutečně dlouho. Proto jsme skript pustili na Google Colab a na výsledky se můžete podívat [zde](https://colab.research.google.com/drive/1B7zys275xmpPqahPwNvuYMPLmgvlV3l5) nebo [zde](/notebooks/07_rbf_rnn/results.txt).